In [ ]:
# TODO:
# Process -> Graph -> Communities -> Extract GNN embs + EchoGAE embs -> ECS
    

In [29]:
import importlib
import src.utils.config_loader
importlib.reload(src.utils.config_loader)

from src.utils.config_loader import ConfigLoader

config_loader = ConfigLoader()
all_configs = config_loader.load_configs()
base_configs = config_loader.get_section(all_configs, "base")
graph_configs = config_loader.get_section(all_configs, "graph")
community_cfg = config_loader.get_section(all_configs, "community")

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}") # type: ignore

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

PyTorch version: 2.8.0+cu128
CUDA available: False
CUDA version: 12.8


device(type='cpu')

In [3]:
# Preprocess data
import importlib
import src.modules.data_processor
importlib.reload(src.modules.data_processor)

from src.modules.data_processor import DataProcessor

print(base_configs)
data_processor = DataProcessor(base_configs=base_configs)
processed_data = data_processor.run(device=device, summarize=True)

{'paths': {'raw': 'data/raw/deb_label.csv', 'processed': 'data/processed'}, 'labels': {0: 'disagree', 1: 'neutral', 2: 'agree'}, 'subreddits': {'brexit': 0, 'blacklivesmatter': 1, 'climate': 2, 'democrats': 3, 'republican': 4}, 'required_columns': ['label', 'msg_id_parent', 'msg_id_child', 'submission_id', 'body_parent', 'body_child', 'submission_text', 'subreddit', 'author_parent', 'author_child', 'datetime', 'agreement_fraction', 'individual_kappa'], 'cleaning': {'normalize_subreddits': True, 'rename_columns': {'author_child': 'src_author', 'author_parent': 'dst_author', 'msg_id_child': 'src_comment_id', 'msg_id_parent': 'dst_comment_id', 'body_child': 'src_comment_text', 'body_parent': 'dst_comment_text', 'datetime': 'timestamp'}, 'timestamp_parsing': {'primary_format': '%d/%m/%Y %H:%M', 'dayfirst': True, 'error_handling': 'coerce', 'fallback_formats': ['%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M', '%Y-%m-%d']}, 'remove_self_replies': True}, 'temporal': {'infer_parent_comment_time': {'enab

,subreddit,Earliest Date,Latest Date,# Replies,% Self-Replies,% Disagree,% Neutral,% Agree,# Unique Comments,# Unique Authors,Total Timesteps,Avg Timestep Window Size (days)
0,All,2015-01-03,2021-05-19,42770,0.0,39.6,26.0,34.4,75034,27420,77,69.3
1,blacklivesmatter,2020-05-27,2021-05-19,1916,0.0,32.9,22.4,44.7,3453,2691,10,33.5
2,brexit,2016-06-21,2021-05-18,15703,0.0,41.6,29.2,29.2,28039,1400,23,76.0
3,climate,2015-01-03,2021-05-18,5754,0.0,40.6,27.7,31.7,10131,5377,26,87.0
4,democrats,2020-01-02,2021-05-19,9606,0.0,35.9,21.9,42.2,16499,8187,9,55.0
5,republican,2020-01-02,2021-05-19,9791,0.0,40.7,24.6,34.7,16912,10154,9,54.9


Subreddit timestep info for replies:
Subreddit: blacklivesmatter (replies)


,timestep,interval,actual_window_size,total_count
0,All,2020-05-27 - 2021-05-19,357,1916
1,0,2020-05-27 - 2020-06-25,29,604
2,1,2020-06-26 - 2020-07-25,29,328
3,2,2020-07-26 - 2020-08-24,29,174
4,3,2020-08-25 - 2020-09-23,29,183
5,4,2020-09-24 - 2020-10-23,29,167
6,5,2020-10-24 - 2020-11-22,29,140
7,6,2020-11-23 - 2020-12-22,28,71
8,7,2020-12-23 - 2021-01-21,29,77
9,8,2021-01-23 - 2021-03-17,53,61


Subreddit: brexit (replies)


...

,timestep,interval,actual_window_size,total_count
0,All,2015-01-03 - 2021-05-18,2327,10131
1,0,2015-01-03 - 2015-04-02,88,220
2,1,2015-04-03 - 2015-07-01,85,205
3,2,2015-07-02 - 2015-09-29,88,233
4,3,2015-09-30 - 2015-12-28,87,201
5,4,2015-12-29 - 2016-03-27,89,206
6,5,2016-03-28 - 2016-06-25,85,181
7,6,2016-06-26 - 2016-09-23,87,222
8,7,2016-09-24 - 2016-12-22,89,392
9,8,2016-12-23 - 2017-03-22,86,273


Subreddit: democrats (comments)


,timestep,interval,actual_window_size,total_count
0,All,2020-01-02 - 2021-05-19,503,16499
1,0,2020-01-02 - 2020-03-01,59,1936
2,1,2020-03-02 - 2020-04-30,59,2402
3,2,2020-05-01 - 2020-06-29,59,2476
4,3,2020-06-30 - 2020-08-28,59,1920
5,4,2020-08-29 - 2020-10-27,59,1931
6,5,2020-10-28 - 2020-12-26,59,2365
7,6,2020-12-27 - 2021-02-24,59,1658
8,7,2021-02-25 - 2021-04-25,59,1155
9,8,2021-04-26 - 2021-05-19,23,656


Subreddit: republican (comments)


,timestep,interval,actual_window_size,total_count
0,All,2020-01-02 - 2021-05-19,503,16912
1,0,2020-01-02 - 2020-03-01,59,810
2,1,2020-03-02 - 2020-04-30,59,1316
3,2,2020-05-01 - 2020-06-29,59,1995
4,3,2020-06-30 - 2020-08-28,59,2399
5,4,2020-08-29 - 2020-10-27,59,2937
6,5,2020-10-28 - 2020-12-26,59,3289
7,6,2020-12-27 - 2021-02-24,59,1448
8,7,2021-02-25 - 2021-04-25,59,1730
9,8,2021-04-26 - 2021-05-19,22,988


In [4]:
# Build graphs
import importlib
import src.modules.graph_processor
importlib.reload(src.modules.graph_processor)

from src.modules.graph_processor import GraphProcessor

processed_path = base_configs.get('processed_path', 'data/processed')
pairs = processed_data.user_pairs
comments = processed_data.comments

graph_processor = GraphProcessor(graph_configs=graph_configs, processed_path=processed_path)
graph_data = graph_processor.run(pairs=pairs, embeddings_source=comments)

Building node features with pooling: mean
    + Total unique authors in pairs: 35257
    + Total pooled vectors: 35212
    + Pooled vector dimension: 384
Building graph snapshots: directed=True, wcc_mode=topk_coverage, edge_attrs=['mean_confidence', 'net_vector']
[Subreddit 0, T0] [WCC:topk_coverage] Nodes 32->29, Edges 86->84 (2.3% edges removed)
[Subreddit 0, T1] [WCC:topk_coverage] Nodes 30->30, Edges 79->79 (0.0% edges removed)
[Subreddit 0, T2] [WCC:topk_coverage] Nodes 31->31, Edges 59->59 (0.0% edges removed)
[Subreddit 0, T3] [WCC:topk_coverage] Nodes 43->43, Edges 107->107 (0.0% edges removed)
[Subreddit 0, T4] [WCC:topk_coverage] Nodes 53->53, Edges 134->134 (0.0% edges removed)
[Subreddit 0, T5] [WCC:topk_coverage] Nodes 71->71, Edges 266->266 (0.0% edges removed)
[Subreddit 0, T6] [WCC:topk_coverage] Nodes 79->79, Edges 187->187 (0.0% edges removed)
[Subreddit 0, T7] [WCC:topk_coverage] Nodes 96->94, Edges 177->176 (0.6% edges removed)
[Subreddit 0, T8] [WCC:topk_coverage] 

c:\Users\vduch\OneDrive\ecs_msc_project\src\modules\graph_processor.py:177: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  x = torch.tensor(node_features, dtype=torch.float)


    + Created 77 PyG graphs
Saved PyG graphs to data/processed/pyg_graphs_384D.pt


In [30]:
# Run community detection
import importlib
import src.modules.community_processor
importlib.reload(src.modules.community_processor)

from src.modules.community_processor import LeidenCommunityProcessor

graph_dict = graph_data.graph_dict

print(community_cfg)
processed_path = base_configs.get('processed_path', 'data/processed')
leiden_processor = LeidenCommunityProcessor(community_configs=community_cfg, output_dir=processed_path)

# Run optimization analysis
print("=============================================================")
print("Optimization Analysis for Community Detection")
print("=============================================================")
opt_meta, scans_df, best_df, summary_df = leiden_processor.analyze_optimal_community_parameters(graph_dict)
print("Optimization meta:", opt_meta)
print("\nBest resolutions per graph (head):")
display(best_df.head() if best_df is not None else "None")
print("Resolution summary:")
display(summary_df if summary_df is not None else "None")

# Community detection results using best parameters
print("=============================================================")
print("Community Detection Results using Best Parameters")
print("=============================================================")
community_result = leiden_processor.run_community_detection(
    graph_dict=graph_dict,
    use_optimization=True,
    save=True
)

print("\nPartitions:")
display(community_result.partitions)
print("Meta:")
print(community_result.meta)

{'algorithm': 'leiden', 'seed': 42, 'weights': {'strategy': 'echo_intensity'}, 'optimization': {'mode': 'per_graph', 'metric': 'modularity', 'min_communities': 3, 'min_community_size': 10, 'max_communities_factor': 0.7, 'resolution': {'grid': [0.05, 0.1, 0.2, 0.4, 0.6, 0.9, 1.0], 'early_stop': {'window': 3, 'delta': 0.001}}}, 'algorithms': {'leiden': {'partition_type': 'RBConfigurationVertexPartition'}}}
Optimization Analysis for Community Detection
<class 'leidenalg.VertexPartition.RBConfigurationVertexPartition'>
Optimization meta: {'mode': 'per_graph', 'algorithm': 'leiden', 'partition_type': 'RBConfigurationVertexPartition', 'weight_strategy': 'echo_intensity', 'uniform_choice': None, 'uniform_scores': [], 'res_grid': [0.05, 0.1, 0.2, 0.4, 0.6, 0.9, 1.0]}

Best resolutions per graph (head):


,subreddit_id,timestep,resolution,num_nodes,num_edges,num_communities,min_size,max_size,mean_size,metric_value,modularity
0,0,5,0.9,71,266,4,16,19,17.750,0.287855,0.287855
1,0,7,0.6,94,176,4,19,28,23.500,0.446555,0.446555
2,0,10,0.9,214,555,8,16,32,26.750,0.407017,0.407017
3,0,11,1.0,261,943,8,25,43,32.625,0.331457,0.331457
4,0,12,1.0,330,1414,11,16,45,30.000,0.283790,0.283790


Resolution summary:


,resolution,metric_mean,metric_std,modularity_mean,num_communities_mean,num_communities_min,num_communities_max,community_size_mean,community_size_min,community_size_max
0,0.05,0.105212,0.046202,0.105212,4.333333,3,6,207.916667,11,833
1,0.10,0.504759,0.199972,0.504759,4.800000,3,6,113.623333,11,702
2,0.20,0.721033,0.078990,0.721033,7.800000,3,11,69.018810,11,228
3,0.40,0.758852,0.072301,0.758852,11.500000,4,17,48.373651,11,177
4,0.60,0.668541,0.191730,0.668541,12.800000,4,22,39.031831,11,98
5,0.90,0.430801,0.205146,0.430801,11.384615,4,25,33.992774,10,74
6,1.00,0.350772,0.062103,0.350772,9.750000,8,11,26.082418,12,45


Community Detection Results using Best Parameters
<class 'leidenalg.VertexPartition.RBConfigurationVertexPartition'>
Saved partitions to data/processed\communities/partitions.parquet
Saved labels arrays to data/processed\communities/labels_arrays.npz
Saved index mapping to data/processed\communities/labels_index.json
Saved name mapping to data/processed\communities/labels_name.json

Partitions:


,subreddit_id,timestep,resolution_used,num_nodes,num_edges,num_communities,modularity,community_sizes
0,0,0,1.0,29,84,5,0.242560,"[9, 9, 6, 3, 2]"
1,0,1,1.0,30,79,5,0.327752,"[8, 6, 6, 5, 5]"
2,0,2,1.0,31,59,6,0.402183,"[7, 5, 5, 5, 5, 4]"
3,0,3,1.0,43,107,6,0.312997,"[12, 10, 7, 7, 4, 3]"
4,0,4,1.0,53,134,6,0.337464,"[12, 11, 9, 8, 7, 6]"
...,...,...,...,...,...,...,...,...
72,4,4,1.0,1195,1260,60,0.878796,"[90, 63, 62, 47, 47, 44, 41, 41, 41, 38, 37, 3..."
73,4,5,1.0,1317,1426,47,0.870551,"[78, 56, 56, 50, 50, 48, 47, 43, 42, 40, 40, 3..."
74,4,6,1.0,657,626,61,0.927661,"[35, 34, 32, 29, 28, 27, 26, 25, 25, 24, 24, 2..."
75,4,7,1.0,719,760,32,0.880666,"[56, 44, 43, 39, 38, 37, 37, 30, 29, 28, 27, 2..."


Meta:
{'algorithm': 'leiden', 'partition_type': 'RBConfigurationVertexPartition', 'weight_strategy': 'echo_intensity', 'use_optimization': True, 'force_resolution': None, 'optimization_meta': {'mode': 'per_graph', 'algorithm': 'leiden', 'partition_type': 'RBConfigurationVertexPartition', 'weight_strategy': 'echo_intensity', 'uniform_choice': None, 'uniform_scores': [], 'res_grid': [0.05, 0.1, 0.2, 0.4, 0.6, 0.9, 1.0]}}


In [32]:
community_result.partitions.num_communities.unique()

array([  5,   6,   4,   7,   9,   8,  11,  13,  10, 119,  77,  57,  46,
        44,  32,  29,  39,  30,  26,  12,  15,  25,  14,  19,  16,  20,
        28,  38,  21,  23,  22,  24,  47,  60,  61])

In [6]:
# Load trained model and extract GNN node embeddings
import importlib
import src.utils.gnn_checkpointing
importlib.reload(src.utils.gnn_checkpointing)

from src.utils.gnn_checkpointing import load_model_checkpoint

ckpt_path = "checkpoints/best_model_2508152029.pth"
model_class_path = "src.models.multitask_debate_gnn.MultitaskDebateGNN"
gnn_model, gnn_ckpt = load_model_checkpoint(
    ckpt_path,
    device=device,
    model_class_path=model_class_path,
)

gnn_model.eval()
print(f"Loaded: {gnn_model.__class__.__name__}")
print(f"Checkpoint keys: {list(gnn_ckpt.keys())}")

Loading model checkpoint from checkpoints/best_model_2508152029.pth
Loaded: MultitaskDebateGNN
Checkpoint keys: ['state_dict', 'model_args', 'train_args']


In [7]:
pyg_graphs = graph_data.pyg_graphs
graph_dict = graph_data.graph_dict
pyg_node_map = graph_data.pyg_node_map

comm_parts = getattr(community_result, "partitions", {})

import numpy as np

for i in pyg_graphs:
    sub = i.subreddit_id if hasattr(i, 'subreddit_id') else None
    ts = i.local_timestep if hasattr(i, 'local_timestep') else None
    print(f"Processing graph for subreddit: {sub}, timestep: {ts}")

    # Get PyG node mapping
    node_map = pyg_node_map[(sub, ts)] if sub is not None and ts is not None and (sub, ts) in pyg_node_map else None
    
    # Map to graph dict for directed graph
    if sub is not None and ts is not None:
        nx_graph = graph_dict[sub][ts]
        assert nx_graph.number_of_nodes() == i.num_nodes, "Node count mismatch"
        assert nx_graph.number_of_edges() == i.num_edges, "Edge count mismatch"        
    else:
        print(f"Skipping graph - missing attributes: subreddit_id={sub}, local_timestep={ts}")

    # Extract node features
    node_features = i.x.cpu().numpy() if hasattr(i, 'x') and i.x is not None else None
    
    # Get GNN node embeddings
    gnn_embeddings = gnn_model.embed(i, device=device, eval_mode=True) if gnn_model else None   # type: ignore

    # Get community data
    # First assert all nodes in pyg map and comm map are the same
    if sub is not None and ts is not None:
        assert i.node_map.keys() == community_result.labels_name_dict[sub][ts].keys()
    else:
        print(f"Skipping community assertion - missing attributes: subreddit_id={sub}, local_timestep={ts}")
    # Attach labels array from community data
    if sub is not None and ts is not None:
        comm_labels = community_result.labels_array[sub][ts]
        assert comm_labels.shape[0] == i.num_nodes, "Label count mismatch"


Processing graph for subreddit: 0, timestep: 0
Processing graph for subreddit: 0, timestep: 1
Processing graph for subreddit: 0, timestep: 2
Processing graph for subreddit: 0, timestep: 3
Processing graph for subreddit: 0, timestep: 4
Processing graph for subreddit: 0, timestep: 5
Processing graph for subreddit: 0, timestep: 6
Processing graph for subreddit: 0, timestep: 7
Processing graph for subreddit: 0, timestep: 8
Processing graph for subreddit: 0, timestep: 9
Processing graph for subreddit: 0, timestep: 10
Processing graph for subreddit: 0, timestep: 11
Processing graph for subreddit: 0, timestep: 12
Processing graph for subreddit: 0, timestep: 13
Processing graph for subreddit: 0, timestep: 14
Processing graph for subreddit: 0, timestep: 15
Processing graph for subreddit: 0, timestep: 16
Processing graph for subreddit: 0, timestep: 17
Processing graph for subreddit: 0, timestep: 18
Processing graph for subreddit: 0, timestep: 19
Processing graph for subreddit: 0, timestep: 20
Pr

In [40]:
pyg_graphs[0].node_map.keys()

dict_keys(['APB2710', 'AnomalyNexus', 'ArchbishopMegatronQC', 'Bozata1', 'EthiczGradient', 'Greengoblingogo', 'Heruss100', 'MrPuddington2', 'Petemcfuzzbuzz', 'Prituh', 'ScarletIT', 'SideburnsOfDoom', 'Skraff', 'SniggeringFool', 'StrixTechnica', 'Thezenstalker', 'Tinkers_toenail', 'Xatom', 'amgiecorker', 'aslate', 'blah-blah-blah12', 'firdseven', 'hishnash', 'pikadrew', 'polic293', 'prodmerc', 'tweeglitch', 'uberdavis', 'yacc143'])

In [52]:
community_result.labels_name_dict[0][0].keys()

dict_keys(['APB2710', 'AnomalyNexus', 'ArchbishopMegatronQC', 'Bozata1', 'EthiczGradient', 'Greengoblingogo', 'Heruss100', 'MrPuddington2', 'Petemcfuzzbuzz', 'Prituh', 'ScarletIT', 'SideburnsOfDoom', 'Skraff', 'SniggeringFool', 'StrixTechnica', 'Thezenstalker', 'Tinkers_toenail', 'Xatom', 'amgiecorker', 'aslate', 'blah-blah-blah12', 'firdseven', 'hishnash', 'pikadrew', 'polic293', 'prodmerc', 'tweeglitch', 'uberdavis', 'yacc143'])

In [53]:
assert pyg_graphs[0].node_map.keys() == community_result.labels_name_dict[0][0].keys()